## Goals

To create an algorithm to predict the presence of heart disease based on the values of 13 features. 

## Import libraries and data

In [5]:
# general libraries
import pandas as pd
import pickle as pkl

# pipeline functions
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV

# classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression

# useful additional functions
from sklearn.model_selection import train_test_split
import re
from sklearn.model_selection import StratifiedKFold

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import TransformerMixin
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import learning_curve


In [9]:
# read pickled df 
df = pd.read_pickle('extended_heart_disease.pkl')


# split data
features = list(df.columns)
features.remove('target')

#split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features]
                                                    , df['target']
                                                    , test_size = 0.2
                                                    , random_state = 42
                                                    , stratify = df['target'])

# read valuable lists
with open('useful_lists.pkl', 'rb') as f:
    valuable_lists = pkl.load(f)

num_features = valuable_lists['num_features']
cat_features = valuable_lists['cat_features']
products = valuable_lists['products']
divisions = valuable_lists['divisions']

## Establishing baseline performance

Before we begin trying to develop an optimised classifier we should establish a base line to compare improvement against.
We will be using f1-scoring to control for any effect of the higher proportion of negatives in our data set.

In [10]:
parameters = {'min_samples_leaf' : (0.01, 0.05, 0.1, 0.2), 'random_state' : [42]}

# the baseline should be taken on the original features, before any additions were made
original_features = ['thal'] + num_features + cat_features 

gscv = GridSearchCV(RandomForestClassifier(), parameters, cv=5, scoring = 'f1')
gscv.fit(X_train[original_features], y_train)

pd.DataFrame(gscv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.222015,0.013017,0.024064,0.003835,0.01,42,"{'min_samples_leaf': 0.01, 'random_state': 42}",0.785714,0.852459,0.830189,0.897959,0.846154,0.842495,0.036234,4
1,0.223750,0.017291,0.024223,0.001864,0.05,42,"{'min_samples_leaf': 0.05, 'random_state': 42}",0.813559,0.866667,0.846154,0.901961,0.851852,0.856038,0.028780,2
2,0.204341,0.005271,0.022147,0.001015,0.1,42,"{'min_samples_leaf': 0.1, 'random_state': 42}",0.793103,0.852459,0.846154,0.880000,0.857143,0.845772,0.028704,3
3,0.197707,0.012857,0.021875,0.001136,0.2,42,"{'min_samples_leaf': 0.2, 'random_state': 42}",0.827586,0.852459,0.846154,0.901961,0.857143,0.857061,0.024596,1


Ok so based on a very simple Grid Search we can take a baseline of 0.857

## Pipeline design

Qualities that we require from the pipeline:
- categorical features must be encoded to ensure that they are not treated numerically
- numerical features, where possible, should be scaled
- we should have flexibility in using different algorithms/feature subsets within the same grid search

In [11]:
# for transforming numerical features
numeric_transformer = StandardScaler()

# for transforming categorical features
# We have encoded categorical features so that, e.g. 1 and 2 are not considered more similar than 1 and 3 when all 3 values just encode different categories.
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

# create ColumnTransformer to transform categorical and numerical features separately
preprocessor = ColumnTransformer(
    transformers =[
        ('num', numeric_transformer, selector(dtype_exclude="category")),
        ('cat', categorical_transformer, selector(dtype_include="category"))
    ]
)

In [12]:
# this enables us to try different classifiers within the same grid search
# taken from the user cgnorthcutt on stackoverflow
class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = RandomForestClassifier(),
        ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [13]:
# this class is designed to enable switching between the features that are used within the same grid search
class FeatureSelector(BaseEstimator):

    def __init__(self, chosen_features = None):
        self.chosen_features = chosen_features

    def fit(self, X, y=None, **kwargs):
        return self

    def transform(self, X, y=None):
        X = X[self.chosen_features]
        return X

## Exploring performance on feature subsets

### Setup

In [14]:
# create different subsets of features to test performance on
original_features =  ['thal'] + num_features + cat_features

feature_subsets = (
    original_features
    , original_features + divisions
    , original_features + products
    , original_features + divisions + products
)

### Initial gridsearch for classifiers

Let's begin by just taking a look at 3 simple classifiers and see how they perform when given a small scope of variation in their hyper-parameters.
We will use grid-search to explore different instances of each classifier.

In [15]:
pipeline = Pipeline([
    ('selector', FeatureSelector())
    , ('preprocessor', preprocessor)
    , ('clf', ClfSwitcher())
])

parameters = [{
        'clf__estimator' : [RandomForestClassifier(random_state = 42)]
        , 'clf__estimator__class_weight' : ('balanced', None)
        , 'selector__chosen_features' : feature_subsets
        , 'clf__estimator__min_samples_leaf' : (0.02, 0.05, 0.1, 0.2, 0.5)
    }
    
    ,{
       'clf__estimator' : [LogisticRegression(random_state = 42)]
        , 'clf__estimator__class_weight' : ('balanced', None)
        , 'selector__chosen_features' : feature_subsets
        , 'clf__estimator__C' : (1.0, 0.3, 0.1, 0.03, 0.01)
    }
    
    ,{
       'clf__estimator' : [SVC(random_state = 42)]
        , 'clf__estimator__class_weight' : ('balanced', None)
        , 'selector__chosen_features' : feature_subsets
        , 'clf__estimator__C' : (1.0, 0.3, 0.1, 0.03, 0.01)
    }
]

Here we are varying a number of different parameters.
Firstly, we are testing with different algorithms.  
Secondly, we are testing with balancing the weights in our cost function, in cases where class_weight = 'balanced', the cost function will take into account the fact that there are slightly more cases where heart disease is present than cases where it is not.
Finally, we are testing different subsets of the features that we have brought through from the expanded feature set. This will allow us to measure the impact of including features and help us establish whether to include them within our algorithm. 

In [16]:
gscv = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1')
gscv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('selector', FeatureSelector()),
                                       ('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000026461C960A0>),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000...
                                                         'trestbps', 'chol',
                                                         'thalach', 'oldpeak',
                   

### Exploring results

#### Presenting results

In order to perform analysis on the results of the grid search it is easier for us to format the results as a data frame.

In [17]:
gs1_results = pd.DataFrame(gscv.cv_results_)

In [19]:
def renaming_features(x):
    '''
    The purpose of this function is to receive one of 4 specified features sets (as a list) and to return a string, naming that feature set
    '''
    if x == original_features:
        return 'original features'
    elif x == original_features + divisions:
        return 'original features + divisions'
    elif x == original_features + products:
        return 'original features + products'
    elif x == original_features + divisions + products:
        return 'all'

In [20]:
# add a column for the feature subset name
gs1_results['features'] = gs1_results['param_selector__chosen_features'].apply(lambda x: renaming_features(x))

In [21]:
# separate out instances of different algorithms
rf1_results = gs1_results.loc[0 : 39].copy()
logres1_results = gs1_results.loc[40 : 79].copy()
svm1_results = gs1_results.loc[80 : 119].copy()

#### Top performers

First of all let us look at the top performing of all the classifier instances.

In [22]:
gs1_results.sort_values('mean_test_score', inplace = True, ascending = False)
gs1_results.head(10)[['param_clf__estimator', 'features', 'mean_test_score']]

,param_clf__estimator,features,mean_test_score
53,"LogisticRegression(C=0.3, random_state=42)",original features + divisions,0.862649
49,"LogisticRegression(C=0.3, random_state=42)",original features + divisions,0.862327
52,"LogisticRegression(C=0.3, random_state=42)",original features,0.861140
54,"LogisticRegression(C=0.3, random_state=42)",original features + products,0.861140
61,"LogisticRegression(C=0.3, random_state=42)",original features + divisions,0.859593
40,"LogisticRegression(C=0.3, random_state=42)",original features,0.858844
28,RandomForestClassifier(random_state=42),original features,0.858572
55,"LogisticRegression(C=0.3, random_state=42)",all,0.858205
48,"LogisticRegression(C=0.3, random_state=42)",original features,0.857455
24,RandomForestClassifier(random_state=42),original features,0.857179


So whilst the top 10 is predominantly composed of LogisticRegression algorithms, the top 2 are both RandomForest.
Performance is not noteworthily better than baseline performance, particularly given the number of instances that we are testing here.
120 instances were tested and the performance of the 10th best is only just above that of the baseline.

#### Random Forest

In [23]:
rf1_results.sort_values('mean_test_score', inplace = True, ascending = False)
rf_columns = ['features', 'param_clf__estimator__class_weight', 'param_clf__estimator__min_samples_leaf', 'mean_test_score', 'std_test_score']
rf1_results.head(10)[rf_columns]

,features,param_clf__estimator__class_weight,param_clf__estimator__min_samples_leaf,mean_test_score,std_test_score
28,original features,None,0.1,0.858572,0.057828
24,original features,None,0.05,0.857179,0.030151
32,original features,None,0.2,0.855353,0.051857
20,original features,None,0.02,0.846730,0.023988
12,original features,balanced,0.2,0.844993,0.053216
8,original features,balanced,0.1,0.834522,0.046426
30,original features + products,None,0.1,0.826956,0.031229
4,original features,balanced,0.05,0.826784,0.040586
21,original features + divisions,None,0.02,0.825236,0.049661
26,original features + products,None,0.05,0.823015,0.033650


Random forest is consistently performing best when only given the original features to train on. All 8 cases where the algorithm is just trained on those features are returned in the top 10.

#### Logistic Regression

In [24]:
logres1_results.sort_values('mean_test_score', inplace = True, ascending = False)
logres_columns = ['features', 'param_clf__estimator__class_weight', 'param_clf__estimator__C', 'mean_test_score', 'std_test_score']
logres1_results.head(10)[logres_columns]

,features,param_clf__estimator__class_weight,param_clf__estimator__C,mean_test_score,std_test_score
53,original features + divisions,None,0.3,0.862649,0.041714
49,original features + divisions,balanced,0.3,0.862327,0.050393
52,original features,None,0.3,0.861140,0.044026
54,original features + products,None,0.3,0.861140,0.044026
61,original features + divisions,None,0.1,0.859593,0.043718
40,original features,balanced,1.0,0.858844,0.046527
55,all,None,0.3,0.858205,0.048744
48,original features,balanced,0.3,0.857455,0.037908
41,original features + divisions,balanced,1.0,0.855676,0.049281
62,original features + products,None,0.1,0.855018,0.035780


The range of performance within the top 10 logisitc regression options is a lot smaller than that seen with Random Forest.
There is also a lot more variety in the features being used in each instance of the algorithm and no one set of features seems dominant.
The most consistent feature is that 0.3 seems to be the optimal value of C.

#### SVM

In [25]:
svm1_results.sort_values('mean_test_score', inplace = True, ascending = False)
svm_columns = ['features', 'param_clf__estimator__class_weight', 'param_clf__estimator__C', 'mean_test_score', 'std_test_score']
svm1_results.head(10)[svm_columns]

,features,param_clf__estimator__class_weight,param_clf__estimator__C,mean_test_score,std_test_score
84,original features,None,1.0,0.840156,0.056151
80,original features,balanced,1.0,0.828256,0.028627
86,original features + products,None,1.0,0.824934,0.035799
92,original features,None,0.3,0.822939,0.031733
88,original features,balanced,0.3,0.822182,0.034039
96,original features,balanced,0.1,0.814604,0.044908
100,original features,None,0.1,0.811379,0.041196
82,original features + products,balanced,1.0,0.803524,0.036047
87,all,None,1.0,0.803038,0.048812
83,all,balanced,1.0,0.801570,0.050821


SVM is performing far below the level of random forest and logistic regression.
The algorithm is generally performing better when trained on just the original features.

#### Overarching patterns

Although a small number of instances of the classifiers have outperformed the baseline, the vast majority have not.
In fact the highest performing instance in the gridsearch was one of the 3 instances considered in the baseline.
This indicates that nothing we have done so far has achieved any improvement on the baseline.

From here on I shall not be using balanced class_weight as in most side-by-side comparisons, balancing the class weights actually reduced performance (measured on f1-score).

If I were performing a more extended analysis I would explore whether this reduction is due to the fact that test_score is evaluated as a simple accuracy percentage, therefore meaning that f1-score might be better when class_weight = 'balanced', however I will not go into this here.

## Trying to improve performance of individual algorithms

### Removing scaling and one hot encoding

The majority of the instances of algorithms tested above performed worse than the baseline which included no scaling/feature encoding. We will explore here whether this processing of the features is actually harming our algorithms. 

#### Random Forest

In [26]:
pipeline = Pipeline([
    ('selector', FeatureSelector())
    , ('rf', RandomForestClassifier(random_state = 42))
])

parameters = [{
        'selector__chosen_features' : feature_subsets
        , 'rf__min_samples_leaf' : (0.02,0.05, 0.1, 0.2)
    }
]

gscv = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1')
gscv.fit(X_train, y_train)

rf_no_scaling_results = pd.DataFrame(data=gscv.cv_results_['mean_test_score'].reshape(4, 4), columns=["all", "originals", "originals + divisors", "originals + products"], index=["0.02", "0.05", "0.1", "0.2"])
rf_no_scaling_results

,all,originals,originals + divisors,originals + products
0.02,0.840616,0.813648,0.841341,0.808004
0.05,0.856038,0.811671,0.831829,0.812376
0.1,0.845772,0.802818,0.829333,0.811743
0.2,0.857061,0.814149,0.787879,0.796578


Ok so performance here is pretty similar to that that seen above.
this makes sense given that random forests are unaffected by feature scaling, however there could be a difference made by the OneHotEncoding.

#### Logistic Regression

In [27]:
pipeline = Pipeline([
    ('selector', FeatureSelector())
    , ('logres', LogisticRegression(random_state = 42))
])

parameters = [{
        'selector__chosen_features' : feature_subsets
        , 'logres__C' : (1.0, 0.3, 0.1, 0.03, 0.01)
    }
]

gscv = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1')
gscv.fit(X_train, y_train)

logres_no_scaling_results = pd.DataFrame(data=gscv.cv_results_['mean_test_score'].reshape(5, 4), columns=["all", "originals", "originals + divisors", "originals + products"], index=["1.0", "0.3", "0.1", "0.03", "0.01"])
logres_no_scaling_results

C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredr\anaconda3\envs\financial_project\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

,all,originals,originals + divisors,originals + products
1.0,0.853731,0.863295,0.761192,0.761280
0.3,0.861138,0.860518,0.764127,0.761055
0.1,0.857350,0.857511,0.765636,0.764127
0.03,0.834224,0.834224,0.764243,0.756639
0.01,0.786869,0.797796,0.765636,0.762779


As a result of no longer scaling the data we get cases where the algorithm fails to converge.
Interestingly here, performance has dropped much more in the cases where we include the products. For cases where these are not included, the performance only changes minimally.

#### SVM

In [28]:
pipeline = Pipeline([
    ('selector', FeatureSelector())
    , ('svm', SVC(random_state = 42))
])

parameters = [{
        'selector__chosen_features' : feature_subsets
        , 'svm__C' : (1.0, 0.3, 0.1, 0.03, 0.01)
    }
]

gscv = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1')
gscv.fit(X_train, y_train)

svm_no_scaling_results = pd.DataFrame(data=gscv.cv_results_['mean_test_score'].reshape(5, 4), columns=["all", "originals", "originals + divisors", "originals + products"], index=["1.0", "0.3", "0.1", "0.03", "0.01"])
svm_no_scaling_results

,all,originals,originals + divisors,originals + products
1.0,0.705711,0.709652,0.705832,0.705832
0.3,0.705832,0.708859,0.705832,0.705832
0.1,0.705832,0.705832,0.679702,0.704367
0.03,0.705832,0.705832,0.705832,0.670580
0.01,0.697987,0.705832,0.705832,0.705832


Performance here has dropped off significantly.
Although SVM performance was below that of the other algorithms in the initial testing it has dropped off significantly here.
This seems reasonable given that, by not scaling the features, the importance of features will be somewhat correlated to the variance of the values of those features.

#### Conclusions

It looks as though in the case of Random Forest, there is at most a minimal difference caused by the scaling and encoding, perhaps because of the encoding of categorical features making separation of values in these cases easier.
As one would expect, the removing of scaling had a much more pronounced impact on the performance of the other two algorithms.

Given these results, we will continue using encoding and feature scaling going forward.

### Exploring different feature subsets

#### Feature Importances with Random Forest

##### Original features

In [29]:
def feature_importance_generator(features):
    pipeline = Pipeline([
        ('preprocessor', preprocessor)
        , ('rf', RandomForestClassifier(min_samples_leaf = 0.02))
    ])
    
    '''
    This function will allow us to generate feature importances for our instances of random forests.
    '''

    clf = pipeline.fit(X_train[features], y_train)
    
    # generate list of post-processing feature names
    num_scale_features =  preprocessor.transformers_[0][2]
    cat_one_hot_features = preprocessor.transformers_[1][1].get_feature_names(cat_features)
    feature_names = num_scale_features + list(cat_one_hot_features)
    
    #collate feature importances and order
    feature_importances = list(zip(feature_names, list(clf.steps[-1][-1].feature_importances_)))

    for feature, importance in sorted(feature_importances, key=lambda x: x[1], reverse = True):
        if importance > 0.02:
            print(feature, ":", importance)

In [30]:
feature_importance_generator(original_features)

cp_0 : 0.16850592746439225
thal : 0.11758132407196134
oldpeak : 0.11078863102188798
ca_0 : 0.08067781556041208
thalach : 0.07944634145752016
exang_0 : 0.06638383656684561
exang_1 : 0.057309784401724254
slope_2 : 0.05418272648559401
age : 0.045352903624305815
chol : 0.03811753821134368
trestbps : 0.03118497814389147
slope_1 : 0.029647727129299094
sex_1 : 0.02462467802404727
cp_2 : 0.021961365226846916


The results here are not as easy to interpret as we might like them to be.
The importance of the categorical features is separated between the different encoded versions of the features and by virtue of there being more features related to (e.g. cp) than there are for numerical features, they are more likely to be selected in the random set of features that can be used to at each node and therefore be attributed more importance.

Having said that, this is a good position to start from when comparing later results.

#### Generating an optimal feature set from scratch

Let's trial performance when we only feed in the more powerful features and see if this helps in any way. Importantly we will see if performance is consistently improved by the addition of these features.

##### Helper function

In [40]:
def feature_evaluater(start_features, all_features, classifier, X_train, y_train, no_improvement_limit = 1):
    """
    Trains a classifier on X_train and y_train using an increasing subset of the features to identify the optimal features
    to use from within the set.
    Returns a df of the features added each time and the performance.
    No_improvement_limit specifies the number of features that can be added in a row without performance improving.
    """
    
    # adjust all_features so that it now contains just the features that we want to test for inclusion
    for feature in start_features:
        all_features.remove(feature)
        
    # rename the list
    non_included_features = all_features
    
    # create a new name for start_features that will be returned as the list of optimum features 
    current_features = start_features
    
    round_count = 1
    
    # create counter for number of features that have been added without improving performance
    no_improvement_count = 0
    
    # set baseline for top score thus far, this variable will represent the top socre achieved thus far
    top_score = 0.01
    
    # create list to keep track of scores
    scores = []
    
    # create list to keep track of features added
    features = []
    
    # create list to keep track of best_features
    best_features = start_features.copy()

    # loop for as long as we do not exceed the limit of added feature without improvement
    while no_improvement_count < no_improvement_limit:
        # create all the subsets to test
        new_feature_subsets = tuple([current_features + [feature] for feature in all_features])

        
        pipeline = Pipeline([
        ('selector', FeatureSelector())
        , ('preprocessor', preprocessor)
        , ('clf', classifier)
        ])

        parameters = {
                'selector__chosen_features' : new_feature_subsets
            }

        gscv_99 = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1')
        gscv_99.fit(X_train, y_train)

        gs99_results = pd.DataFrame(gscv_99.cv_results_)
        
        new_scores = gs99_results['mean_test_score']
        new_feature_index = new_scores.idxmax()
        new_features = gs99_results['param_selector__chosen_features'][new_feature_index]
        score = new_scores[new_feature_index]
        
        # add the score to scores
        scores.append(score)
        
        # add the new feature to features
        features.append(new_features[-1])
        
        
        current_features = new_features
        
        # remove the now included feature from non-included features
        non_included_features.remove(new_features[-1])
        round_count += 1

        # test for at least minimal improvement
        if score - top_score > 0.0005:
            no_improvement_count = 0
            top_score = score
            best_features = new_features

        else:
            no_improvement_count += 1
        
    feature_scores = pd.DataFrame(list(zip(features, scores)),
               columns =['Added feature', 'Score'])
    
    return feature_scores, best_features

##### Random Forest

Given that cp was the most important feature identified in the feature importances, let us take this as the first feature. 

In [38]:
rf_feature_scores, rf_best_features = feature_evaluater(['cp']
                                  , original_features + products + divisions
                                  , RandomForestClassifier(random_state = 42, min_samples_leaf = 0.2)
                                  , X_train
                                  , y_train
                                  , no_improvement_limit = 3)

That is the best performance that we have seen so far from a single algorithm.
The previous best performance that we had seen from a RF was 0.858, so performance here has increased by over 1.5%. 

In [41]:
rf_feature_scores

,Added feature,Score
0,oldpeak / age,0.821049
1,thal,0.844953
2,ca,0.856466
3,exang,0.865270
4,slope,0.873985
5,oldpeak / chol,0.872355
6,age * thalach,0.870178
7,thalach,0.875368
8,trestbps * thalach,0.874736
9,restecg,0.869393


In [43]:
rf_best_features

['cp',
 'oldpeak / age',
 'thal',
 'ca',
 'exang',
 'slope',
 'oldpeak / chol',
 'age * thalach',
 'thalach']

##### Logistic Regression

In [46]:
log_res_feature_scores, log_res_best_features = feature_evaluater([] ## do not give any features to begin with as we have nothing to work from 
                                              , original_features + products + divisions
                                              , LogisticRegression(random_state = 42, C = 0.2)
                                              , X_train
                                              , y_train
                                              , no_improvement_limit = 5) ##training is a lot quicker with LogRes so giving more scope for improvement

In [47]:
log_res_feature_scores

,Added feature,Score
0,thal,0.789757
1,thalach / chol,0.798406
2,oldpeak * oldpeak,0.807593
3,cp,0.845465
4,chol * thalach,0.854392
5,oldpeak / thalach,0.863319
6,chol,0.863319
7,restecg,0.862672
8,trestbps / chol,0.863698
9,chol * oldpeak,0.866865


This is also the best performance that we have seen from Logistic Regression, going from 0.862 to 0.867.

##### SVM

In [50]:
svm_feature_scores, svm_best_features = feature_evaluater([] ## do not give any features to begin with as we have nothing to work from
                                      , original_features + products + divisions
                                      , SVC(random_state = 42, C = 1.0)
                                      , X_train
                                      , y_train
                                      , no_improvement_limit = 3)

In [51]:
svm_feature_scores

,Added feature,Score
0,thal,0.796013
1,ca,0.810086
2,exang,0.832785
3,thalach / chol,0.847621
4,oldpeak / trestbps,0.850847
5,chol / thalach,0.850343
6,age * age,0.856790
7,chol * chol,0.857029
8,sex,0.856787
9,chol * thalach,0.866942


Performance here far outstrips that seen with any other instance of SVC and in fact that top performing feature subset here outperforms that of LogisticRegression.
Previously the top performance seen for SVC was 0.840.

##### Analysis

With more time it would be a lot better to use different random seeds to generate a more varied set of results, from which we could draw a set of features that consistently occur in the optimal feature sets.
Confidence in these results could also have been improved by varying the hyper-parameters to test a more varied set of circumstances.

In [55]:
print(rf_best_features, '\n')
print(log_res_best_features, '\n')
print(svm_best_features, '\n')

['cp', 'oldpeak / age', 'thal', 'ca', 'exang', 'slope', 'oldpeak / chol', 'age * thalach', 'thalach'] 

['thal', 'thalach / chol', 'oldpeak * oldpeak', 'cp', 'chol * thalach', 'oldpeak / thalach', 'chol', 'restecg', 'trestbps / chol', 'chol * oldpeak'] 

['thal', 'ca', 'exang', 'thalach / chol', 'oldpeak / trestbps', 'chol / thalach', 'age * age', 'chol * chol', 'sex', 'chol * thalach', 'chol'] 



It is interesting to see that despite being the most important feature for the random forest algorithm, cp does not even get used by the svm model.
Secondly, the ratios and products are more prominent for the log_res and svm models than they are in rf.

## Error analysis

Let's try to look to see how common the misclassifications are between the algorithms.
If the overlap between errors is minimal then we can conclude that it might be possible to train an ensemble classifier to correctly classify these cases. However if all 3 algorithms are getting particular cases wrong then these issues would not be addressed by any ensemble of the classifiers.

### rf and logres

In [377]:
def error_comparison(classifier_list, classifier_names_list, classifier_features_list, X_train, y_train):
    """
    function to compare the errors of different classifiers on the same data set
    """
    
    # split the data into 5 folds
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(X_train, y_train)
    
    result = {}
    
    # identify the overlap in errors for each split of the data
    for train_index, test_index in skf.split(X_train, y_train):
        
        # store the results for each individual data split
        temp_results_df = pd.DataFrame()
        
        # identify the samples to be used in the split
        X_train_k, X_test_k = X_train.iloc[train_index].copy(), X_train.iloc[test_index].copy()
        y_train_k, y_test_k = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
        
        # add the correct classifications to the data frame
        temp_results_df['y_test_k'] = y_test_k
        
        # find the predicted classes for each algorithm
        for clf, clf_name, feature_list in list(zip(classifier_list, classifier_names_list, classifier_features_list)):
             

            # create the pipeline
            pipeline = Pipeline([
            ('selector', FeatureSelector(feature_list))
            , ('preprocessor', preprocessor)
            , (clf_name, clf)
            ])
            
        
            # fit pipeline
            pipeline.fit(X_train_k[feature_list], y_train_k)
            
            # identify class predictions
            predictions = pipeline.predict(X_test_k[feature_list])
            
            # add results to the data frame
            temp_results_df[clf_name] = predictions
            
            # add prediction accuracy to dict
            prior_results = result.get(clf_name, [])
            prior_results.append((predictions == y_test_k).mean())
            
            result[clf_name] = prior_results
        
        # create a copy of the classifiers_name_list to remove each classifier once it has been used
        names_copy = classifier_names_list.copy()
        
        for clf_name_1 in classifier_names_list:
            names_copy.remove(clf_name_1)
            
            for clf_name_2 in names_copy:
            
                new_field_name = clf_name_1 + ' or ' + clf_name_2
                
                #get predictions for clfs
                predictions_1 = temp_results_df[clf_name_1]
                predictions_2 = temp_results_df[clf_name_2]
                
                #add prediction accuracy to dict
                prior_results = result.get(new_field_name, [])
                
                prior_results.append(((predictions_2 == y_test_k) | (predictions_1 == y_test_k)).mean())
            
                result[new_field_name] = prior_results  
                
    final_results = pd.DataFrame(result)
    
    return final_results

In [378]:
classifier_list = [RandomForestClassifier(random_state = 42, min_samples_leaf = 0.2)
                   , LogisticRegression(random_state = 42, C = 0.3)
                  , SVC(random_state = 42, C = 1.0)]
classifier_names_list = ['rf', 'log_res', 'svm']
classifier_features_list = [rf_features, log_res_features, svm_features]

resutls_df = error_comparison(classifier_list, classifier_names_list, classifier_features_list, X_train, y_train)

In [379]:
resutls_df

,rf,log_res,svm,rf or log_res,rf or svm,log_res or svm
0,0.816327,0.816327,0.795918,0.836735,0.877551,0.897959
1,0.775510,0.795918,0.897959,0.836735,0.897959,0.897959
2,0.875000,0.791667,0.833333,0.895833,0.916667,0.895833
3,0.937500,0.916667,0.895833,0.958333,0.958333,0.916667
4,0.833333,0.875000,0.854167,0.916667,0.895833,0.916667


In [161]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X_train, y_train)

5

In [163]:
combined_scores = pd.DataFrame(list(zip(correct_predictions_1, correct_predictions_2, either_correct)),columns =['rf', 'logres', 'either'])
combined_scores['improvement'] = combined_scores['either'] - combined_scores[['rf', 'logres']].max(axis = 1)
combined_scores

,rf,logres,either,improvement
0,0.755102,0.734694,0.795918,0.040816
1,0.795918,0.836735,0.836735,0.000000
2,0.833333,0.854167,0.916667,0.062500
3,0.854167,0.895833,0.895833,0.000000
4,0.875000,0.895833,0.916667,0.020833


Ok so there is a minimal amount of improvement here. This suggests that a model that combined the predictions of both models in order to decide on which class to predict could improve by up to 4%. However there is no reason to believe that it would. In general the majority of cases that one algorithm gets wrong, the other algorithm also predicts incorrectly.
Let us just see if this is any different when using a third algorithm.

### rf and svm

In [169]:
correct_predictions_1 = []
correct_predictions_2 = []
either_correct = []

pipeline1 = Pipeline([
        ('preprocessor', preprocessor)
        , ('rf', RandomForestClassifier(class_weight = 'balanced', min_samples_leaf = 0.2))
    ])

pipeline2 = Pipeline([
    ('preprocessor', preprocessor)
    , ('svm', SVC(class_weight = 'balanced', C = 0.3))
    ])


for train_index, test_index in skf.split(X_train, y_train):
    
    X_train_k, X_test_k = X_train.iloc[train_index].copy(), X_train.iloc[test_index].copy()
    y_train_k, y_test_k = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
    
    # pipeline 1
    pipeline1.fit(X_train_k[cont_features + disc_features], y_train_k)
    
    predictions_1 = pipeline1.predict(X_test_k[cont_features + disc_features])
    
    correct_predictions_1.append((predictions_1 == y_test_k).mean())
    
    # pipeline 2
    pipeline2.fit(X_train_k[cont_features + disc_features], y_train_k)
    
    predictions_2 = pipeline2.predict(X_test_k[cont_features + disc_features])
    
    correct_predictions_2.append((predictions_2 == y_test_k).mean())
    
    
    either_correct.append(((predictions_2 == y_test_k) | (predictions_1 == y_test_k)).mean())

In [170]:
combined_scores = pd.DataFrame(list(zip(correct_predictions_1, correct_predictions_2, either_correct)),columns =['rf', 'svm', 'either'])
combined_scores['improvement'] = combined_scores['either'] - combined_scores[['rf', 'svm']].max(axis = 1)
combined_scores

,rf,svm,either,improvement
0,0.755102,0.775510,0.795918,0.020408
1,0.836735,0.816327,0.897959,0.061224
2,0.875000,0.812500,0.916667,0.041667
3,0.875000,0.833333,0.875000,0.000000
4,0.895833,0.875000,0.937500,0.041667


Ok in cases where performance is poor there is some potential gain.

I don't understand why performance is generally performing as we go through the models. This seems unlikely to be a coincidence but I cannot identify what could be causing it.

### Trying with 3 models

In [171]:
correct_predictions_1 = []
correct_predictions_2 = []
correct_predictions_3 = []
best_of_3_correct = []

pipeline1 = Pipeline([
        ('preprocessor', preprocessor)
        , ('rf', RandomForestClassifier(class_weight = 'balanced', min_samples_leaf = 0.2))
    ])

pipeline2 = Pipeline([
    ('preprocessor', preprocessor)
    , ('svm', SVC(class_weight = 'balanced', C = 0.3))
    ])

pipeline3 = Pipeline([
        ('preprocessor', preprocessor)
        , ('logres', LogisticRegression(class_weight = 'balanced', C = 1.0))
    ])

for train_index, test_index in skf.split(X_train, y_train):
    
    X_train_k, X_test_k = X_train.iloc[train_index].copy(), X_train.iloc[test_index].copy()
    y_train_k, y_test_k = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
    
    # pipeline 1
    pipeline1.fit(X_train_k[cont_features + disc_features], y_train_k)
    
    predictions_1 = pipeline1.predict(X_test_k[cont_features + disc_features])
    
    correct_predictions_1.append((predictions_1 == y_test_k).mean())
    
    # pipeline 2
    pipeline2.fit(X_train_k[cont_features + disc_features], y_train_k)
    
    predictions_2 = pipeline2.predict(X_test_k[cont_features + disc_features])
    
    correct_predictions_2.append((predictions_2 == y_test_k).mean())
    
    # pipeline 3
    pipeline3.fit(X_train_k[cont_features + disc_features], y_train_k)
    
    predictions_3 = pipeline3.predict(X_test_k[cont_features + disc_features])
    
    correct_predictions_3.append((predictions_3 == y_test_k).mean())
    
    best_of_3 = ((predictions_2 == y_test_k) & (predictions_1 == y_test_k)) | ((predictions_3 == y_test_k) & (predictions_1 == y_test_k)) | ((predictions_3 == y_test_k) & (predictions_2 == y_test_k))
    
    best_of_3_correct.append(best_of_3.mean())

In [174]:
combined_scores = pd.DataFrame(list(zip(correct_predictions_1, correct_predictions_2, correct_predictions_3, best_of_3_correct)),columns =['rf', 'svm', 'logres', 'best_of_3'])
combined_scores['improvement'] = combined_scores['best_of_3'] - combined_scores[['rf', 'svm', 'logres']].max(axis = 1)
combined_scores

,rf,svm,logres,best_of_3,improvement
0,0.714286,0.775510,0.734694,0.775510,0.000000
1,0.795918,0.816327,0.836735,0.836735,0.000000
2,0.875000,0.812500,0.854167,0.833333,-0.041667
3,0.854167,0.833333,0.895833,0.854167,-0.041667
4,0.875000,0.875000,0.895833,0.916667,0.020833


Ok so without any probabilistic elements this looks just as likely to have a negative impact as it is to have a positive impact.

### Findings

In general the cases that are being predicted incorrectly are quite consistent across the algorithms that we have been trying. For this reason it is worth trying something like xgboost which specifically looks to bolster performance by targeting cases that are being predicted incorrectly. 

## XGBoost

In [182]:
pipeline = Pipeline([
    ('preprocessor', preprocessor)
    , ('classifier', XGBClassifier(use_label_encoder=False))
])

parameters = {
    'classifier__max_depth' : [3, 6, 9]
    , 'classifier__eta' : [0.1, 0.3, 0.5]
}

gscv = GridSearchCV(pipeline, parameters, scoring= 'f1')

gscv.fit(X_train, y_train)

[22:28:38] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:38] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:38] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:38] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[22:28:43] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:43] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:43] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:28:44] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002024BF445B0>),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002024BF44FA0>)])),
                                       ('classifier',
                                        XGBC...
                                                      missing=nan,
                                                      monotone_constraints=None,
 

In [183]:
gscv.cv_results_

{'mean_fit_time': array([0.13787932, 0.15771422, 0.16086793, 0.12515678, 0.13286543,
        0.13350229, 0.11890798, 0.12167983, 0.12245426]),
 'std_fit_time': array([0.00401977, 0.00595612, 0.00100928, 0.00391245, 0.00259745,
        0.00161635, 0.00549176, 0.00081128, 0.0018578 ]),
 'mean_score_time': array([0.01497755, 0.01471515, 0.01553082, 0.01586776, 0.0154449 ,
        0.01576281, 0.01517587, 0.01510777, 0.01536317]),
 'std_score_time': array([0.0004538 , 0.00107665, 0.00063866, 0.00075124, 0.00056636,
        0.00061837, 0.00120851, 0.00060906, 0.00081501]),
 'param_classifier__eta': masked_array(data=[0.1, 0.1, 0.1, 0.3, 0.3, 0.3, 0.5, 0.5, 0.5],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_classifier__max_depth': masked_array(data=[3, 6, 9, 3, 6, 9, 3, 6, 9],
              mask=[False, False, False, False, False, False, False, False,
                    Fals

This has not initially proved any more useful than other algorithms.

## Unused material that may prove useful

In [29]:
# set up a train, validation set for use
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42, stratify = y_train)

## Messing

In [265]:
feature_subset = original_features + products + divisions
feature_subset.remove('thal')
feature_subset.remove('cp')

set_features = ['thal', 'cp']
round_count = 1

top_score = 0.8

while True:
    new_feature_subsets = tuple([set_features + [feature] for feature in feature_subset])
    
    pipeline = Pipeline([
    ('selector', FeatureSelector())
    , ('preprocessor', preprocessor)
    , ('clf', ClfSwitcher())
    ])

    parameters = [{
            'clf__estimator' : [RandomForestClassifier(random_state = 42, min_samples_leaf = 0.2)]
            , 'selector__chosen_features' : new_feature_subsets
        }

        ,{
           'clf__estimator' : [LogisticRegression(random_state = 42, C = 0.3)]
            , 'selector__chosen_features' : new_feature_subsets
        }

        ,{
           'clf__estimator' : [SVC(random_state = 42, C = 1.0)]
            , 'selector__chosen_features' : new_feature_subsets
        }
    ]
    
    gscv_99 = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'f1')
    gscv_99.fit(X_train, y_train)
    
    gs99_results = pd.DataFrame(gscv_99.cv_results_)
    
    gs99_results['features'] = gs99_results['param_selector__chosen_features'].apply(lambda x: tuple(x))
    
    scores = gs99_results.groupby('features')['mean_test_score'].mean()
    new_features = scores.idxmax()
    score = scores[new_features]
    print(round_count, new_features, score)
    
    percent_increase = (score - top_score) / top_score
    
    if percent_increase > 0.001:
        top_score = score
        set_features = list(new_features)
        feature_subset.remove(new_features[-1])
        round_count += 1
    
    else:
        break